# Imports used for the project

In [2]:
# -*- coding: utf-8 -*-
import csv
import datetime
from cleantext import clean
import re
import numpy as np
import pandas as pd

import psycopg2
from psycopg2 import connect, extensions, sql, Error
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

import scrapy
from scrapy.crawler import CrawlerProcess

import sys
import os

# Task 2 - Load database

- Demonstrate that you have a working database containing the FakeNewsCorpus dataset.

- Explain your choice of schema design. 

- Demonstrate that your database contains a larger number of rows (e.g. one million - or however many you can reasonably work with on your available hardware)

- Demonstrate that your database supports simple queries.

### Create csv files that contains the data from the raw data csv file

In [3]:
# Sample
csv_in = '1mio-raw.csv' #You need to have the file in the same folder as this Jupyter Notebook

In [4]:
# Initialize the dictionaries
author_dictionary  = dict()
domain_dictionary  = dict() 
type_dictionary    = dict()
keyword_dictionary = dict()   
article_dictionary = dict()

In [5]:
def create_empty_file_for_writing(filename):
    if os.path.exists(filename):
      os.remove(filename)
    return open(filename, "w")    

In [6]:
csv_author     = create_empty_file_for_writing('author.csv')
csv_keyword    = create_empty_file_for_writing('keyword.csv')
csv_domain     = create_empty_file_for_writing('domain.csv')
csv_type       = create_empty_file_for_writing('type.csv')
csv_article    = create_empty_file_for_writing('article.csv')
csv_written_by = create_empty_file_for_writing('written_by.csv')
csv_tags       = create_empty_file_for_writing('tags.csv')
csv_webpage    = create_empty_file_for_writing('webpage.csv')

In [7]:
def put_in_dictionary(dictionary, liste, ID):
    dictionary.clear()
    liste = list(set(liste)) # By converting the list to a set we sort the list and remove duplicates
    for j in range(len(liste)):
        info = str(liste[j])
        dictionary[info] = ID
        ID += 1
    return ID    

In [8]:
def simple_entity_to_CSV(filename, idName, keyName, dictionary):
    file = open(filename,"a",encoding='utf-8')
    for item in dictionary.items():
        file.write("%s,%s\n" %(str(item[1]), str(item[0])))
    file.close

In [23]:
from nltk.tokenize import word_tokenize
import string
from nltk.corpus import stopwords
def clean_text(content):

    # split into words
    tokens = word_tokenize(content)
    
    # convert to lower case
    tokens = [w.lower() for w in tokens]
    
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    
    # remove remaining tokens that are not alphabetic
    words = [word for word in stripped if word.isalpha()]
    
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    clean_text = [w for w in words if not w in stop_words]
    
    #Join to one string
    seperator = ' '
    clean_text = seperator.join(clean_text)
    """
    # Set all words to be lowercased
    clean_text = content.lower()
    
    # Clean dates 
    date1 = r"\b(?:jan(?:uary|\.)?|feb(?:ruary|\.)?|mar(?:ch|\.)?|apr(?:il|\.)?|may|jun(?:e|\.)?|jul(?:y|\.)?|aug(?:ust|\.)?|sep(?:tember|\.)?|oct(?:ober|\.)?|(nov|dec)(?:ember|\.)?) (?:\d{1,2})(,)? (?:1\d{3}|2\d{3})(?=\D|$)"
    date2 = r"\b(\d{1, 2})? (?:jan(?:uary|\.)?|feb(?:ruary|\.)?|mar(?:ch|\.)?|apr(?:il|\.)?|may|jun(?:e|\.)?|jul(?:y|\.)?|aug(?:ust|\.)?|sep(?:tember|\.)?|oct(?:ober|\.)?|(nov|dec)(?:ember|\.)?) (?:1\d{3}|2\d{3})(?=\D|$)"
    date3 = r"\b(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|(nov|dec)(?:ember)?) (of)? (?:(?:\d{1,2})|(?:1\d{3}|2\d{3}))(?=\D|$)"                         
    date4 = r"(?:\d{1,2}/\d{1,2}/\d{4}|\d{1,2}-\d{1,2}-\d{4})"
    date_patterns = [date1, date2, date3, date4]
    
    for pattern in date_patterns:
        clean_text = re.sub(pattern, ' <DATE> ', clean_text)
    
    # Clean email
    email1 = r'([\w0-9._-]+@[\w0-9._-]+\.[\w0-9_-]+)'
    clean_text = re.sub(email1, ' <EMAIL> ', clean_text)
    
    # Clean URLs 
    url1 = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    clean_text = re.sub(url1, ' <URL> ', clean_text)
    
    # Clean numbers
    num1 = r'[0-9]+'
    clean_text = re.sub(num1, ' <NUM> ', clean_text)
    
    # Clean multiple white spaces, tabs, and newlines
    space1 = r"\s+"
    clean_text = re.sub(space1, ' ', clean_text)
    clean_text = clean_text.replace('\n', '')
    
    # Remove ^ since we use them as delimiter
    clean_text = clean_text.replace('^', '')
    
    # Remove "
    clean_text = clean_text.replace('"', '')
    
    # Remove { since we use them as delimiter
    clean_text = clean_text.replace('{', '')
    clean_text = clean_text.replace('}', '')
    """
    return clean_text

In [24]:
def isNaN(string):
    return string != string

In [27]:
ID_author = ID_domain = ID_keyword = ID_type = ID_article = 0
def extract_and_put_in_csv_files(chunk):
    
    temp1 = []
    temp2 = []
    temp3 = []
    temp4 = []
    
    global ID_author
    global ID_domain
    global ID_keyword
    global ID_type 
    global ID_article
    
    
    for index, row in chunk.iterrows():
        # Authors  
        authors = row['authors']
        if(not isNaN(authors)):
            list_of_authors = authors.split(", ")
            for author in list_of_authors:
                if(len(author) <= 64):
                    temp1.append(author)    
        
        # Meta keywords
        keywords = row['meta_keywords']
        if(not isNaN(keywords)):
            list_of_keywords = re.split(r'[;,"\'\[\]]\s*', keywords)
            for keyword in list_of_keywords:
                if(len(keyword) <= 128):
                    temp2.append(keyword)
        
        # Domain
        domain = row['domain']
        if(not isNaN(domain)):
            if(len(domain) <= 1024):
                temp3.append(domain)
        
        # Type
        typ = row['type']
        if(not isNaN(typ)):
            if(len(typ) <= 64):
                temp4.append(typ)
        
    # Authors    
    ID_author = put_in_dictionary(author_dictionary, temp1, ID_author)
    #with open('author.csv', 'w', newline='') as authorfile:
    #    fieldnames = ['author_id', 'author_name']
    #    writer = csv.DictWriter(authorfile, fieldnames=fieldnames)
    #authorfile.close()
    simple_entity_to_CSV("author.csv", "author_id", "author_name", author_dictionary)
    
    # Meta keywords
    ID_domain = put_in_dictionary(keyword_dictionary, temp2, ID_domain)
    simple_entity_to_CSV("keyword.csv", "keyword_id", "keyword", keyword_dictionary)
    
    # Domain
    ID_keyword = put_in_dictionary(domain_dictionary, temp3, ID_keyword)
    simple_entity_to_CSV("domain.csv", "domain_id", "domain_url", domain_dictionary)
    
    # Type
    ID_type = put_in_dictionary(type_dictionary, temp4, ID_type)
    simple_entity_to_CSV("type.csv", "type_id", "type_name", type_dictionary)
        

        
    for index, row in chunk.iterrows():
        # Article
        title = row['title']
        if(title and (not isNaN(title)) and (len(title) <= 512)):
            title = clean_text(title)
        else:
            title = "NULL"
        
        content = row['content']
        if(not isNaN(content)):
            content = clean_text(content)
        else:
            content = "NULL"
        
        summary = row['summary']
        if(summary and (not isNaN(summary))):
            summary = clean_text(summary)
        else:
            summary = "NULL"
        
        meta_description = row['meta_description']
        if(meta_description and (not isNaN(meta_description))):
            meta_description = clean_text(meta_description)
        else:
            meta_description = "NULL"
        
        typ = row['type']
        if((not isNaN(typ)) and (len(typ) <= 64)):
            type_id = type_dictionary[typ]
        else:
            type_id = 0
        
        scrapped_at = row['scraped_at']
        if(isNaN(scrapped_at)):
            scrapped_at = datetime.datetime(1000, 1, 1)
        inserted_at = row['inserted_at']
        if(isNaN(inserted_at)):
            inserted_at = datetime.datetime(1000, 1, 1)
        updated_at  = row['updated_at']
        if(isNaN(updated_at)):
            updated_at = datetime.datetime(1000, 1, 1)
        
        #
        #ID_author = put_in_dictionary(author_dictionary, )
        
        #print(f"^%s^%s^%s\n" % (title, content, meta_description))
        #csv_article = open("article.csv","a",encoding='utf-8')
        csv_article.write("%s^\"%s\"^\"%s\"^\"%s\"^\"%s\"^%s^%s^%s^%s\n"%
            (ID_article
             , title.encode(encoding='utf_8', errors='replace')#.decode(encoding='utf_8', errors='ignore')
             , content.encode(encoding='utf_8', errors='replace')#.decode(encoding='utf_8', errors='ignore')
             , summary.encode(encoding='utf_8', errors='replace')#.decode(encoding='utf_8', errors='ignore')
             , meta_description.encode(encoding='utf_8', errors='replace')#.decode(encoding='utf_8', errors='ignore')
             , type_id
             , scrapped_at
             , inserted_at
             , updated_at))
        #csv_article.close()
        
        # Webpage
        url = row['url']
        domain = row['domain']
        if((not isNaN(url)) and (not isNaN(domain)) and (len(url) <= 1024) and (len(domain) <= 1024)):
            #csv_webpage = open("webpage.csv","a",encoding='utf-8')
            csv_webpage.write("%s^%s^%s\n" % (ID_article, url, domain_dictionary[domain]))
            #csv_webpage.close()
        
        # Writen_by
        authors = row['authors']
        if(not isNaN(authors)):
            list_of_authors = authors.split(", ")
            for author in list_of_authors:
                if(len(author) <= 64):
                    #csv_written_by = open("written_by.csv","a",encoding='utf-8')
                    csv_written_by.write("%s,%s\n" % (ID_article, author_dictionary[author]))
                    #csv_written_by.close()
        
        # Tags
        keywords = row['meta_keywords']
        if(not isNaN(keywords)):
            keywords = re.split(r'[;,"\'\[\]]\s*', keywords)
            for keyword in keywords:
                if(len(keyword) <= 128):
                    #csv_tags = open("tags.csv","a",encoding='utf-8')
                    csv_tags.write("%s,%s\n" % (ID_article, keyword_dictionary[keyword]))
                    #csv_tags.close()
        
        ID_article += 1


In [28]:
csv_author.close()     
csv_keyword.close()
csv_domain.close()
csv_type.close()       
csv_article.close()    
csv_written_by.close() 
csv_tags.close()       
csv_webpage.close()
csv_author     = create_empty_file_for_writing('author.csv')
csv_keyword    = create_empty_file_for_writing('keyword.csv')
csv_domain     = create_empty_file_for_writing('domain.csv')
csv_type       = create_empty_file_for_writing('type.csv')
csv_article    = create_empty_file_for_writing('article.csv')
csv_written_by = create_empty_file_for_writing('written_by.csv')
csv_tags       = create_empty_file_for_writing('tags.csv')
csv_webpage    = create_empty_file_for_writing('webpage.csv')
# Process and store the stuff 
CHUNK_SIZE = 100000 # read 100k rows at a time

with open(csv_in, 'r', newline='',encoding='utf-8', errors='replace') as csvfile:
    reader = pd.read_csv(csvfile, chunksize=CHUNK_SIZE, encoding='utf-8',parse_dates=['scraped_at','inserted_at','updated_at'])
    """, dtype={"Unnamed: 0": int64, "id": int64, "domain": object,
                                                              "type": object,"url": object,"content": object,
                                                              "scraped_at": object,"inserted_at": object,"updated_at": object,
                                                              "title": object, "authors": object,"meta_keywords": object,
                                                              "meta_description": object,"tags": object ,"summary": object,
                                                              "source": object}"""
    chunk_idx = 1 # Used to see how far the reading process is
    for chunk in reader:
        extract_and_put_in_csv_files(chunk)
        print(str(CHUNK_SIZE * chunk_idx) + " rows have been read.")
        chunk_idx += 1
csvfile.close() 

KeyboardInterrupt: 

### Create database and load it with the data from above

Run the follwoing commands in the terminal to create the database:

sudo -u postgres psql

CREATE DATABASE fakenews;

CREATE USER user123 WITH ENCRYPTED PASSWORD 'pass123';

GRANT ALL PRIVILEGES ON DATABASE FakeNews TO user123;

GRANT pg_read_server_files TO user123;

In [29]:
sql_query_create_and_load_database = """
DROP TABLE IF EXISTS Written_by, Tags, Webpage, Article, Typ, Domain, Author, Keyword CASCADE;


CREATE TABLE Keyword (
  keyword_id serial,
  keyword varchar(128),
  PRIMARY KEY (keyword_id)
);

CREATE TABLE Author (
  author_id serial,
  author_name varchar(64),
  PRIMARY KEY (author_id)
);

CREATE TABLE Domain (
  domain_id serial,
  domain_url varchar(1024),
  PRIMARY KEY (domain_id)
);

CREATE TABLE Typ (
  type_id serial,
  type_name varchar(64),
  PRIMARY KEY (type_id)
);

CREATE TABLE Article (
  article_id integer,
  title text,
  content text,
  summary text,
  meta_description text,
  type_id integer REFERENCES Typ(type_id),
  inserted_at timestamp,
  updated_at timestamp,
  scraped_at timestamp,
  PRIMARY KEY (article_id)
);

CREATE TABLE Webpage (
  article_id integer REFERENCES Article(article_id),
  url varchar(1024),
  domain_id integer REFERENCES Domain(domain_id)
);

CREATE TABLE Tags (
  article_id integer REFERENCES Article(article_id),
  keyword_id integer REFERENCES Keyword(keyword_id)
);

CREATE TABLE Written_by (
  article_id integer REFERENCES Article(article_id),
  author_id integer REFERENCES Author(author_id)
);

COPY keyword    FROM '/Users/Public/Project_Data_Science/keyword.csv'     DELIMITER ',' CSV;
COPY author     FROM '/Users/Public/Project_Data_Science/author.csv'      DELIMITER ',' CSV;
COPY typ        FROM '/Users/Public/Project_Data_Science/type.csv'        DELIMITER ',' CSV;
COPY domain     FROM '/Users/Public/Project_Data_Science/domain.csv'      DELIMITER ',' CSV;
COPY article    FROM '/Users/Public/Project_Data_Science/article.csv'     DELIMITER '^' QUOTE '}' CSV ;
COPY tags       FROM '/Users/Public/Project_Data_Science/tags.csv'      DELIMITER ',' CSV;
COPY written_by FROM '/Users/Public/Project_Data_Science/written_by.csv' DELIMITER ',' CSV;
COPY webpage    FROM '/Users/Public/Project_Data_Science/webpage.csv'   DELIMITER '^' CSV;"""

In [30]:
# Establishing the connection
conn = psycopg2.connect(
   database="fakenews", user='user123', password='pass123', host='127.0.0.1', port= '5432'
)

# Set the isolation level
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Execute SQL commands
cursor.execute(sql_query_create_and_load_database)

# Close the cursor to avoid memory leaks
cursor.close()

# Close the connection to avoid memory leaks
conn.close()

Login to the database to see that the data has been loaded with the following command:

psql -h localhost -U user123 fakenews

### Explanation of the database design 

We have chosen to design our database this way since ...

### Demonstration that our database contains a larger number of rows

In [31]:
sql_query_large_number_of_rows_keyword    = "SELECT COUNT(*) FROM keyword;"
sql_query_large_number_of_rows_author     = "SELECT COUNT(*) FROM author;"
sql_query_large_number_of_rows_domain     = "SELECT COUNT(*) FROM domain;"
sql_query_large_number_of_rows_typ        = "SELECT COUNT(*) FROM typ;"
sql_query_large_number_of_rows_article    = "SELECT COUNT(*) FROM article;"
sql_query_large_number_of_rows_webpage    = "SELECT COUNT(*) FROM webpage;"
sql_query_large_number_of_rows_tags       = "SELECT COUNT(*) FROM tags;"
sql_query_large_number_of_rows_written_by = "SELECT COUNT(*) FROM written_by;"

In [32]:
# Establishing the connection
conn = psycopg2.connect(
   database="fakenews", user='user123', password='pass123', host='127.0.0.1', port= '5432'
)

# Set the isolation level
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Creating a cursor object using the cursor() method
cursor = conn.cursor()

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_keyword)
# Get result from SQL command
print("Number of rows in the keyword table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_author)
# Get result from SQL command
print("Number of rows in the author table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_domain)
# Get result from SQL command
print("Number of rows in the domain table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_typ)
# Get result from SQL command
print("Number of rows in the typ table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_article)
# Get result from SQL command
print("Number of rows in the article table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_webpage)
# Get result from SQL command
print("Number of rows in the webpage table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_tags)
# Get result from SQL command
print("Number of rows in the tags table: " + str(cursor.fetchone()[0]))

# Execute SQL command
cursor.execute(sql_query_large_number_of_rows_written_by)
# Get result from SQL command
print("Number of rows in the written_by table: " + str(cursor.fetchone()[0]))


# Close the cursor to avoid memory leaks
cursor.close()

# Close the connection to avoid memory leaks
conn.close()

Number of rows in the keyword table: 33321
Number of rows in the author table: 10376
Number of rows in the domain table: 246
Number of rows in the typ table: 12
Number of rows in the article table: 21790
Number of rows in the webpage table: 21767
Number of rows in the tags table: 162193
Number of rows in the written_by table: 29723


# Task 3 - Issuing queries to better understand the characteristics of the data


Formulate the following queries in the database languages requested (in the square brackets following each item) and briefly discuss what you observe when you execute them over your database: 

3.1: List the domains of news articles of reliable type and scrapped at or after January 15, 2018. NOTE: Do not include duplicate domains in your answer. [Languages: relational algebra and SQL]

3.2: List the name(s) of the most prolific author(s) of news articles of fake type. An author is among the most prolific if it has authored as many or more fake news articles as any other author in the dataset. [Languages: extended relational algebra and SQL]

3.3: Count the pairs of article IDs that exhibit the exact same set of meta-keywords, but only return the pairs where the set of meta-keywords is not empty. [Language: SQL]

# Task 4 - Explore the FakeNewsCorpus dataset

Make at least three non-trivial observations/discoveries about the data. These observations could be related to outliers, artefacts, or even better: genuinely interesting patterns in the data that could potentially be used for fake-news detection. Examples of simple observations could be how many missing values there are in particular columns - or what the distribution over domains is. Be creative! :). Note that many of these observations can be extracted as direct queries to the database, for instance using GROUP BY and COUNT.

# Task 5 - Creation of our own news data set by scraping it from the web.

We will be looking at the "Politics and Conflict" section of the Wikinews site (https://en.wikinews.org/wiki/Category:Politics_and_conflicts), which contains about 7500 articles sorted by the first letter in their title. Since we want the different groups to have slightly different experiences with this data, each group should try to extract the articles for a specific range of letters - given by the python expression: 

"ABCDEFGHIJKLMNOPRSTUVWZABCDEFGHIJKLMNOPRSTUVWZ"[group_nr%23:group_nr%23+10]

where group_nr is your group number (according to Task 1). 

The data set you produce should contain fields corresponding to the content of the article, in addition to some metadata fields like the date when the article was written. Describe the tools you used, and any challenges that you faced, and report some basic statistics on the data (e.g. number of rows, fields, etc). Note that there are no fake/no-fake labels in this dataset - we will consider it as a trusted source of only true articles (which is perhaps a bit naive).

In [ ]:
class testSpider(scrapy.Spider):
    name = "wikitest"
    def start_requests(self):
        start_urls = [
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=D",
            '''"https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=E",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=F",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=G",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=H",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=I",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=J",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=K",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=L",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=M",
            "https://en.wikinews.org/w/index.php?title=Category:Politics_and_conflicts&from=N",'''
        ]


        #yield all the article links from the start urls
        for url in start_urls:
            yield scrapy.Request( url = url, callback = self.parse )
        
        
    #We want to follow the next page from the start urls until  we are finished entirely (at letter O)
    #Will find all links in the start url that correspond to category
    #Might also find some categories that should not be processed --- Fixed no longer applicable
    def parse(self, response):    
        links = response.xpath('/html/body/div[3]/div[3]/div[4]/div[2]/div[2]/div/div/div/ul/li/a/@href').extract()
        WantedArticles = r"/wiki/[D-N]"
        for link in links:
            if link and (re.match(WantedArticles , link ) != None):
                yield response.follow(url = link, callback = self.parse2)

        #Change the last in the string to the last letter
        forbiddenNP =  r"https:\/\/en\.wikinews\.org\/w\/index\.php\?title=Category:Politics_and_conflicts&pagefrom=O"
        #This has to be get() instead of extract so that forbiddenNP in nextpageurl can be compared as strings
        nextpageurl1 = response.xpath("//*[@id='mw-pages']/a[contains(.,'next page')]/@href").get()
        nextpageurl = response.urljoin(nextpageurl1)

        #Obama signs $787 billion stimulus package #Last on the N page
        #Obama signs healthcare bill for 9/11 emergency workers #first on the O page
        if nextpageurl and (re.match(forbiddenNP , nextpageurl ) == None):
            # If we've found a pattern which matches
            #print("Found url: {}".format(nextpageurl)) # Write a debug statement
            yield scrapy.Request(nextpageurl, callback=self.parse) # Return a call to the function "parse"

    #Will run over all individual articles and extract appropriate data
    def parse2(self, response):
        #return individual article 
        #title
        title = response.xpath('/html/body/div[3]/h1/text()').get()
        #source(s)
        source = response.xpath('/html/body/div[3]/div[3]/div[4]/div/ul/li/span/descendant-or-self::text()').extract()
        #date
        date = response.xpath('/html/body/div[3]/div[3]/div[4]/div/p[1]/strong/text()').get()
        #text
        text = response.xpath("""/html/body/div[3]/div[3]/div[4]/div/p/descendant-or-self::text()[not( parent::strong | ancestor::i)]""").extract()

        finaltext = ''.join(text)

        finalsource = ''.join(source)

        yield {
            'title': title,
            'source' : finalsource,
            'date' : date,
            'text' : finaltext,
        }

        #put date text and source in right side of equal sign
        #title as key
        test_dict[title] = [finalsource , date , finaltext]

#dictionary to store data
test_dict = dict()

#run spider
process = CrawlerProcess()
process.crawl(testSpider)
process.start()

# Print a preview of dict
var = test_dict.keys()
print(var)